# Coffee Roast Prediction Model

<h2>Importing 

In [1]:
import torch
import os
import torch.nn as nn
import torchvision
from torchvision import datasets,transforms
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import cv2
from tqdm.auto import tqdm
import random
from torch.utils.data import DataLoader
import re
from sklearn.metrics import accuracy_score
from torchvision.transforms import v2

<h2> Loading Dataset

In [2]:
device = "cuda"

In [3]:
transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize(size=(224,224), antialias=True)
])
    

In [4]:
path = (r'C:\Users\rohit\Desktop\machine learning\Datasets\coffee\train')
dir = os.listdir(path)
cat = list(dir)
train_fol = torchvision.datasets.ImageFolder(path,transform = transform)
train_load = DataLoader(train_fol,
                        batch_size=32,
                        shuffle=True,
                       )

In [5]:
path1 = (r'C:\Users\rohit\Desktop\machine learning\Datasets\coffee\test')
dir1 = os.listdir(path1)
cat1 = list(dir1)
test_fol = torchvision.datasets.ImageFolder(path1,transform = transform)
test_load = DataLoader(test_fol,
                        batch_size=32,
                        shuffle=True,
                       )

<h2> Model Creation and Training

In [6]:
class mod1(nn.Module):
    def __init__(self,input_size:int,hidden_size:int,output_size:int):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels = input_size,
                      out_channels = hidden_size,
                      kernel_size = (3,3),
                      stride=1,
                      padding="same"),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = (2,2))
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels = hidden_size,
                      out_channels = hidden_size,
                      kernel_size = (3,3),
                      stride=1,
                      padding="same"),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size = (2,2))
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels = hidden_size,
                      out_channels = hidden_size,
                      kernel_size = (3,3),
                      stride=1,
                      padding="same"),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size = (2,2))
        )
        self.block4 = nn.Sequential(
            nn.Conv2d(in_channels = hidden_size,
                      out_channels = hidden_size,
                      kernel_size = (3,3),
                      stride=1,
                      padding="same"),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size = (2,2))
        )
        self.conn = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features = hidden_size*14*14,
                      out_features = hidden_size*14*14),
            nn.LeakyReLU(),
            nn.Linear(in_features = hidden_size*14*14,
                      out_features = hidden_size*14*14),
            nn.LeakyReLU(),
            nn.Linear(in_features = hidden_size*14*14,
                      out_features = output_size),
            nn.LeakyReLU()
        )
        
    def forward(self,x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.conn(x)
        return x

In [7]:
epoch = 10
lr = 0.0006
model = mod1(3,15,4).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = lr)
loss = nn.CrossEntropyLoss()


In [8]:
for i in tqdm(range(epoch)):
    train_loss = 0
    test_accuracy =0 
    test_loss = 0

    for batch,(x,y) in enumerate(train_load):
        model.train()
        x = x.to(device)
        y = y.to(device)
        logit = model(x)
        cost = loss(logit,y)
        train_loss +=cost
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    model.eval()
    with torch.inference_mode():
        for batch,(x,y) in enumerate(test_load):
            x = x.to(device)
            y = y.to(device)
            logit = model(x)
            cost = loss(logit,y)
            test_loss += cost
            acc = accuracy_score(y.cpu(),logit.argmax(dim=1).cpu())
            acc = acc*100
            test_accuracy += acc
        
        
        train_loss = train_loss/len(train_load)
        test_loss = test_loss/len(test_load)
        test_accuracy = test_accuracy/len(test_load)
        print(f"COST PER EPOCH : {train_loss}   TEST LOSS : {test_loss}  TEST ACCURACY : {test_accuracy}")

  0%|          | 0/10 [00:00<?, ?it/s]

COST PER EPOCH : 1.3925727605819702   TEST LOSS : 1.3862990140914917  TEST ACCURACY : 24.51923076923077
COST PER EPOCH : 1.386260986328125   TEST LOSS : 1.3941317796707153  TEST ACCURACY : 25.0
COST PER EPOCH : 1.38828706741333   TEST LOSS : 1.3862981796264648  TEST ACCURACY : 24.51923076923077
COST PER EPOCH : 1.3863240480422974   TEST LOSS : 1.3862683773040771  TEST ACCURACY : 25.721153846153847
COST PER EPOCH : 1.386304497718811   TEST LOSS : 1.38633131980896  TEST ACCURACY : 24.278846153846153
COST PER EPOCH : 1.3862991333007812   TEST LOSS : 1.3861361742019653  TEST ACCURACY : 30.76923076923077
COST PER EPOCH : 1.4028819799423218   TEST LOSS : 1.3838791847229004  TEST ACCURACY : 29.08653846153846
COST PER EPOCH : 1.3408883810043335   TEST LOSS : 1.0898807048797607  TEST ACCURACY : 51.44230769230769
COST PER EPOCH : 0.8362435102462769   TEST LOSS : 0.6699026226997375  TEST ACCURACY : 62.5
COST PER EPOCH : 0.5274962782859802   TEST LOSS : 0.34171319007873535  TEST ACCURACY : 87.9807